# **RC4**

## **Chương trình**

### **KSA**

Đọc khóa từ file

In [1]:
with open("key.bin", "rb") as f:
    key = f.read()
print("Độ dài khóa:", len(key))
print("Khóa:", key)

Độ dài khóa: 7
Khóa: b'cryptii'


Tạo mảng S có độ dài 256 byte được đánh số từ 0 đến 255

In [2]:
S = [i for i in range(256)]
print("Mảng S ban đầu:", S)

Mảng S ban đầu: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218

Trộn khóa

In [3]:
j = 0

for i in range(256):
    # Cập nhật chỉ số j
    j = (j + S[i] + key[i % len(key)]) % 256
    # Hoán đổi S[i] và S[j]
    S[i], S[j] = S[j], S[i]

print("Mảng S sau khi trộn khóa:", S)

Mảng S sau khi trộn khóa: [99, 219, 173, 196, 90, 64, 25, 138, 172, 127, 249, 73, 55, 83, 121, 53, 222, 95, 229, 183, 137, 109, 20, 136, 246, 112, 103, 42, 11, 66, 24, 8, 63, 203, 17, 217, 101, 182, 164, 178, 232, 98, 80, 140, 49, 206, 189, 69, 169, 161, 76, 128, 88, 205, 108, 96, 167, 82, 170, 176, 208, 148, 117, 181, 67, 242, 133, 50, 129, 132, 139, 145, 57, 120, 22, 125, 59, 226, 162, 106, 195, 72, 1, 0, 68, 97, 163, 104, 105, 28, 185, 116, 253, 119, 233, 23, 237, 244, 54, 190, 94, 46, 7, 79, 35, 166, 243, 201, 199, 100, 84, 250, 194, 165, 144, 114, 210, 81, 225, 159, 30, 37, 230, 231, 204, 197, 179, 238, 2, 146, 102, 77, 89, 213, 78, 134, 141, 43, 187, 251, 16, 34, 143, 110, 115, 218, 223, 38, 5, 184, 254, 74, 202, 124, 151, 91, 180, 193, 200, 175, 168, 21, 152, 157, 3, 252, 6, 92, 70, 44, 45, 153, 236, 87, 19, 188, 14, 58, 29, 156, 142, 209, 234, 15, 65, 245, 33, 131, 61, 118, 215, 85, 41, 48, 52, 239, 216, 51, 75, 255, 211, 18, 113, 221, 126, 191, 123, 12, 149, 192, 27, 56, 177, 

### **PRGA**

Đọc bản rõ từ file

In [4]:
with open("plaintext.txt", "rb") as f:
	plaintext = f.read()
	
print("Độ dài bản rõ:", len(plaintext))
print("Bản rõ:", plaintext)

Độ dài bản rõ: 44
Bản rõ: b'The quick brown fox jumps over the lazy dog.'


Tạo mảng để lưu bản mã

In [5]:
ciphertext = []

**Mã hóa**

In [6]:
i = 0
j = 0

for _ in range(len(plaintext)):
    input("Nhấn Enter để xử lý byte tiếp theo...")
    i = (i + 1) % 256
    j = (j + S[i]) % 256
    S[i], S[j] = S[j], S[i]
    K = S[(S[i] + S[j]) % 256]
    cipher = plaintext[_] ^ K
    ciphertext.append(cipher)
    print(f"Byte {_:3}: Bản rõ = {plaintext[_]:3} , K = {K:3} , Bản mã = {cipher}")

Byte   0: Bản rõ =  84 , K = 126 , Bản mã = 42
Byte   1: Bản rõ = 104 , K = 170 , Bản mã = 194
Byte   2: Bản rõ = 101 , K = 155 , Bản mã = 254
Byte   3: Bản rõ =  32 , K = 237 , Bản mã = 205
Byte   4: Bản rõ = 113 , K = 169 , Bản mã = 216
Byte   5: Bản rõ = 117 , K = 142 , Bản mã = 251
Byte   6: Bản rõ = 105 , K = 209 , Bản mã = 184
Byte   7: Bản rõ =  99 , K =  37 , Bản mã = 70
Byte   8: Bản rõ = 107 , K =  83 , Bản mã = 56
Byte   9: Bản rõ =  32 , K = 195 , Bản mã = 227
Byte  10: Bản rõ =  98 , K = 198 , Bản mã = 164
Byte  11: Bản rõ = 114 , K = 240 , Bản mã = 130
Byte  12: Bản rõ = 111 , K =  97 , Bản mã = 14
Byte  13: Bản rõ = 119 , K = 197 , Bản mã = 178
Byte  14: Bản rõ = 110 , K = 107 , Bản mã = 5
Byte  15: Bản rõ =  32 , K = 236 , Bản mã = 204
Byte  16: Bản rõ = 102 , K = 232 , Bản mã = 142
Byte  17: Bản rõ = 111 , K =  70 , Bản mã = 41
Byte  18: Bản rõ = 120 , K = 186 , Bản mã = 194
Byte  19: Bản rõ =  32 , K = 171 , Bản mã = 139
Byte  20: Bản rõ = 106 , K = 247 , Bản mã = 157

In [7]:

with open("ciphertext.bin", "wb") as f:
	f.write(bytes(ciphertext))

Kết quả sau khi mã hóa

In [8]:
print("Độ dài bản mã:", len(ciphertext))
print("Bản mã:", bytes(ciphertext).hex())

Độ dài bản mã: 44
Bản mã: 2ac2fecdd8fbb84638e3a4820eb205cc8e29c28b9d5d6b2ef974f311964971c90e8b9ca16467ef2dc6fc3520


**Giải mã**

Tạo mảng lưu kết quả giải mã

In [9]:
decrypted = []

Tạo lại mảng S vì sau khi mã hóa mảng S đã bị xáo trộn

In [10]:
S = [i for i in range(256)]
print("Mảng S ban đầu:", S)

Mảng S ban đầu: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218

In [11]:
j = 0

for i in range(256):
    # Cập nhật chỉ số j
    j = (j + S[i] + key[i % len(key)]) % 256
    # Hoán đổi S[i] và S[j]
    S[i], S[j] = S[j], S[i]

print("Mảng S sau khi trộn khóa:", S)

Mảng S sau khi trộn khóa: [99, 219, 173, 196, 90, 64, 25, 138, 172, 127, 249, 73, 55, 83, 121, 53, 222, 95, 229, 183, 137, 109, 20, 136, 246, 112, 103, 42, 11, 66, 24, 8, 63, 203, 17, 217, 101, 182, 164, 178, 232, 98, 80, 140, 49, 206, 189, 69, 169, 161, 76, 128, 88, 205, 108, 96, 167, 82, 170, 176, 208, 148, 117, 181, 67, 242, 133, 50, 129, 132, 139, 145, 57, 120, 22, 125, 59, 226, 162, 106, 195, 72, 1, 0, 68, 97, 163, 104, 105, 28, 185, 116, 253, 119, 233, 23, 237, 244, 54, 190, 94, 46, 7, 79, 35, 166, 243, 201, 199, 100, 84, 250, 194, 165, 144, 114, 210, 81, 225, 159, 30, 37, 230, 231, 204, 197, 179, 238, 2, 146, 102, 77, 89, 213, 78, 134, 141, 43, 187, 251, 16, 34, 143, 110, 115, 218, 223, 38, 5, 184, 254, 74, 202, 124, 151, 91, 180, 193, 200, 175, 168, 21, 152, 157, 3, 252, 6, 92, 70, 44, 45, 153, 236, 87, 19, 188, 14, 58, 29, 156, 142, 209, 234, 15, 65, 245, 33, 131, 61, 118, 215, 85, 41, 48, 52, 239, 216, 51, 75, 255, 211, 18, 113, 221, 126, 191, 123, 12, 149, 192, 27, 56, 177, 

Giải mã theo PRGA

In [12]:
i = 0
j = 0
for _ in range(len(ciphertext)):
    input("Nhấn Enter để xử lý byte tiếp theo...")
    i = (i + 1) % 256
    j = (j + S[i]) % 256
    S[i], S[j] = S[j], S[i]
    K = S[(S[i] + S[j]) % 256]
    plain = ciphertext[_] ^ K
    decrypted.append(plain)
    print(f"Byte {_:3}: Bản mã = {ciphertext[_]:3} , K = {K:3} , Bản rõ = {plain}")

with open("decrypted.txt", "wb") as f:
	f.write(bytes(decrypted))

Byte   0: Bản mã =  42 , K = 126 , Bản rõ = 84
Byte   1: Bản mã = 194 , K = 170 , Bản rõ = 104
Byte   2: Bản mã = 254 , K = 155 , Bản rõ = 101
Byte   3: Bản mã = 205 , K = 237 , Bản rõ = 32
Byte   4: Bản mã = 216 , K = 169 , Bản rõ = 113
Byte   5: Bản mã = 251 , K = 142 , Bản rõ = 117
Byte   6: Bản mã = 184 , K = 209 , Bản rõ = 105
Byte   7: Bản mã =  70 , K =  37 , Bản rõ = 99
Byte   8: Bản mã =  56 , K =  83 , Bản rõ = 107
Byte   9: Bản mã = 227 , K = 195 , Bản rõ = 32
Byte  10: Bản mã = 164 , K = 198 , Bản rõ = 98
Byte  11: Bản mã = 130 , K = 240 , Bản rõ = 114
Byte  12: Bản mã =  14 , K =  97 , Bản rõ = 111
Byte  13: Bản mã = 178 , K = 197 , Bản rõ = 119
Byte  14: Bản mã =   5 , K = 107 , Bản rõ = 110
Byte  15: Bản mã = 204 , K = 236 , Bản rõ = 32
Byte  16: Bản mã = 142 , K = 232 , Bản rõ = 102
Byte  17: Bản mã =  41 , K =  70 , Bản rõ = 111
Byte  18: Bản mã = 194 , K = 186 , Bản rõ = 120
Byte  19: Bản mã = 139 , K = 171 , Bản rõ = 32
Byte  20: Bản mã = 157 , K = 247 , Bản rõ = 106

Kết quả sau khi giải mã

In [13]:
print("Độ dài bản mã sau giải mã:", len(decrypted))
print("Bản mã sau giải mã:", bytes(decrypted))

Độ dài bản mã sau giải mã: 44
Bản mã sau giải mã: b'The quick brown fox jumps over the lazy dog.'
